<h1>Ditchley S2DS project August 2020 - Code Pipeline<h1>
    <h2>Team: Adam Hawken, Luca Lamoni, Elizabeth Nicholson, Robert Webster<h2>

In [ ]:
#![]() #graphical representation of the pipeline here

In [2]:
# Import modules and set up working directory
import sys
import os
import time
import logging
import json
import csv
import threading
import queue
import asyncio 
import nest_asyncio
nest_asyncio.apply()
import twint
import pandas as pd

sys.path.insert(1, 'C:/Users/Luca/Aug20_Ditchley')

<h3>Section 1: Getting journalist twitter handles according to a keyword<h3>
    <h4>The journalist scraping is performed at the web address https://www.journalism.co.uk/prof/?chunk=0&cmd=default<h4>

In [ ]:
from src.data import journalists as journos
keyword = 'cybersecurity'
journo_handles = journos.get_handles_by_keyword(keyword)
print(len(journo_handles))

<h3>Section 2. Scrape user information and friend lists for each journalist in the list<h3>
    <h4>Section 2.1: Scrape user information using the Twitter API<h4> 

In [ ]:
import json
import tweepy
from src.data import api_tweepy as api
#Load twitter API credentials and return a tweepy API instance
tw_api = api.connect_API('../src/data/twitter_credentials.json')

In [ ]:
# Scrape user information using the API
from src.data import api_user_tools as api_tools
from src.data import data_cleanup as dc
api_users = api_tools.batch_request_user_info(tw_api,journo_handles)
df_api = dc.populate_user_df(api_users)
# Check
df_api.head()

In [ ]:
# Save the dataframe
df_api.to_csv('../data/raw/'+keyword+'_user_profiles.csv', index = False)

<h4>Section 2.2: Scrape user friend list using Twint<h4> 

In [ ]:
from src.data import twint_tools as tt
# define keyword arguments
kwargs = {'n_retries':5,
         'suppress':False}
# multi threading
tt.twint_in_queue(tt._get_friends, 3, journo_handles, args=('../data/raw/'+keyword+'_',), kwargs=kwargs)

In [ ]:
# Concatenate all the individual lists into one dataframe with journalist and its friends
friends_csv = tt.join_friends_csv(journo_handles,keyword) # this function has a bug, the first friend name is 'username'

In [ ]:
# Save the dataframe
friends_csv.to_csv('../data/raw/'+keyword+'_journalist_friends.csv', index=False)

<h3>Section 3. Loop over selected journalists handles and scrape their tweets (3.1) and mentions (3.2) using Twint<h3>
    <h4>Section 3.1: Scrape tweets using Twint<h4> 

In [ ]:
from src.data import twint_tools as tt
# define keyword arguments
kwargs = {'date_range':('2020-08-01 00:00:00', None),
         'n_retries':5,
         'suppress':False}
# multi threading
tt.twint_in_queue(tt._search_tweets_by_user, 3, journo_handles, args=('../data/raw/'+keyword+'_',), kwargs=kwargs)

In [ ]:
# Joined all the individual csv into one dataframe
cyber_test = tt.join_tweet_csv(journo_handles, keyword)
# Check
cyber_test.head()

In [ ]:
#this comes later


# Standardise the twint output 
from src.data import data_cleanup as dc
#Create the standardized template
test_twint = dc.standard_tweet_dataset_setup('test_twint')
test_twint
#fill the template
standard_tweet_twint = dc.fill_standard_tweet_dataset_with_twint(test_twint, cyber_test)
# Check
standard_tweet_twint.head()

In [ ]:
# Save the dataframe
standard_tweet_twint.to_csv('../data/processed/' + keyword + '_standard_tweets_twint.csv',index=False)

<h4>Section 3.2: Extract mentions from Twint dataset<h4> 

In [ ]:
# from the twint dataset, extract mentions based on tweet id and save in a separate csv
mentions_twint  = dc.twint_mentions_to_df(cyber_test)
# Check
mentions_twint.head()

In [ ]:
# Save the dataframe
mentions_twint.to_csv('../data/processed/' + keyword + '_mentions_twint.csv',index=False)

<h4>Section 2.3: Scrape list of friends for each journalist using Twint  ############# we moved this up the pipeline<h4> 

In [ ]:
from src.data import twint_tools as tt
# define keyword arguments
kwargs = {#'date_range':('2020-08-01 00:00:00', None),
         'n_retries':5,
         'suppress':False}
# multi threading
tt.twint_in_queue(tt._get_friends, 3, journo_handles, args=('../data/raw/'+keyword+'_',), kwargs=kwargs)

In [ ]:
# Concatenate all the individual lists into one dataframe with journalist and its friends
friends_csv = tt.join_friends_csv(journo_handles,keyword) # this function has a bug, the first friend name is 'username'

In [ ]:
# Save the dataframe
friends_csv.to_csv('../data/raw/'+keyword+'_journalist_friends.csv', index=False)

<h4>Section 2.4: Scrape journalist user information using Twitter API ################ this also was moved up the pipeline<h4> 

In [ ]:
import json
import tweepy
from src.data import api_tweepy as api
#Load twitter API credentials and return a tweepy API instance
tw_api = api.connect_API('../src/data/twitter_credentials.json')

In [ ]:
# Scrape user information using the API
from src.data import api_user_tools as api_tools
from src.data import data_cleanup as dc
api_users = api_tools.batch_request_user_info(tw_api,journo_handles)
df_api = dc.populate_user_df(api_users)
# Check
df_api.head()

In [ ]:
# Save the dataframe
df_api.to_csv('../data/raw/'+keyword+'_user_profiles.csv', index = False)

<h3>Section 4. Loop over selected journalists handles and scrape their tweets (4.1) and mentions (4.2) using Twitter API<h3>
    <h4>Section 4.1: Scrape tweets using Twint ################ I am waiting for Rob function here<h4> 

In [ ]:
import json
import tweepy
from src.data import api_tweepy as api
#Load twitter API credentials and return a tweepy API instance
tw_api = api.connect_API('../src/data/twitter_credentials.json')

<h4>Section 3.2: Extract mentions from API tweets<h4> 


<h4>Section 3.3: Scrape list of friends for each journalist using Twint ################### this has been moved up<h4>    

In [ ]:
from src.data import twint_tools as tt
# define keyword arguments
kwargs = {#'date_range':('2020-08-01 00:00:00', None),
         'n_retries':5,
         'suppress':False}
# multi threading
tt.twint_in_queue(tt._get_friends, 3, journo_handles, args=('../data/raw/'+keyword+'_',), kwargs=kwargs)

In [ ]:
# Concatenate all the individual lists into one dataframe with journalist and its friends
friends_csv = tt.join_friends_csv(journo_handles,keyword) # this function has a bug, the first friend name is 'username'

In [ ]:
# Save the dataframe
friends_csv.to_csv('../data/raw/'+keyword+'_journalist_friends.csv', index=False)

<h4>Section 3.4: Scrape journalist user information using Twitter API ################### this has been moved u<h4>

In [ ]:
import json
import tweepy
from src.data import api_tweepy as api
#Load twitter API credentials and return a tweepy API instance
tw_api = api.connect_API('../src/data/twitter_credentials.json')

In [ ]:
# Scrape user information using the API
from src.data import api_user_tools as api_tools
from src.data import data_cleanup as dc
api_users = api_tools.batch_request_user_info(tw_api,journo_handles)
df_api = dc.populate_user_df(api_users)
# Check
df_api.head()

In [ ]:
# Save the dataframe
df_api.to_csv('../data/raw/'+keyword+'_user_profiles.csv', index = False)

<h3>Section 5. data cleaning and standardization/LDA<h3>

<h3>Section 6. Create graph database and import twitter data into it<h3>
    <h4>Section 6.1: Download modules, import modules and load graph database<h4> 

In [ ]:
!pip install py2neo

In [9]:
# import standard libraries
import numpy as np
import pandas as pd
from py2neo import Graph
from py2neo.data import Node, Relationship
from src.data import graphdb as gdb

# load / declare the database
graph = gdb.get_graph(new_graph = True)

In [14]:
# we need to fix the file location



# lowd in user information
print('Loading in user information and drawing (Person) nodes')
fn_users = 'cyber_user_profiles.csv'
gdb.load_users(fn_users ,graph)

Loading in user information and drawing (Person) nodes


ClientError: [Statement.SemanticError] Cannot merge node using null property value for user_description (Failure when processing file '/C:/Users/Luca/.Neo4jDesktop/neo4jDatabases/database-80342547-14db-43df-98c2-7debd0a85816/installation-4.1.0/import/cyber_user_profiles.csv' on line 34.)

<h4>Section 6.2: Load friend information into DB<h4> 

In [ ]:
# load in friend information
print('Loading in friends info and drawing [FOLLOWS] edges')
fn_friends = 'cyber_journalist_friends_2_0.csv'
gdb.load_friends(fn_friends,graph)

<h4>Section 6.3: Load tweet data into DB<h4> 

In [11]:
# load in tweet information
print('Loading in tweets and drawing (Tweet) nodes')
fn_tweets = 'standardised_cyber_tweets_19082020.csv'
gdb.load_tweets(fn_tweets ,graph) 

Loading in tweets and drawing (Tweet) nodes


<h4>Section 6.4: Draw edges between users and their tweets<h4> 

In [ ]:
# draw edges between users and their tweets
print('Drawing [POSTS] edges')
gdb.get_posts(graph)


<h4>Section 6.5: Load tweets' mentions<h4> 

In [ ]:
# load in mentions information
print('Loading in mentions and drawing [MENTIONS] edges')
fn_mentions = 'twint_cyber_16082020_mentions.csv'
gdb.load_mentions(fn_mentions,graph)

<h4>Section 6.6: Run page rank algorithm using [FOLLOWS] [MENTIONS] edges<h4> 

In [ ]:
# run Page rank using follower and mention edges
print('ruuning page rank')
nodelist = ['Person','Tweet']
edgelist = ['FOLLOWS','MENTIONS']
page_rank_friends_mentions = gdb.run_pagerank(nodelist,edgelist,graph)

<h4>Section 6.7: Get a weighted random sample from the journalists friends<h4> 

In [ ]:
# get a weighted random sample of users
n_sample = 20
fields = ['rank']
exponents = [2]
sample = gdb.get_multiple_weighted_sample(page_rank_friends_mentions,n_sample,fields,exponents)